# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


In [3]:
import pandas as pd
import numpy as np


### First, download the data from: https://www.kaggle.com/ntnu-testimon/paysim1. Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?

In [4]:
# Your code here
# file too big - saved in my pc

In [7]:
df = pd.read_csv("../../NTNU_data.csv", nrows = 500000)

In [8]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [9]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,500000.000000,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,500000.000000,500000.0
mean,13.906728,1.663937e+05,9.116928e+05,9.314261e+05,9.827739e+05,1.162668e+06,0.000466,0.0
std,3.731891,2.725841e+05,3.016901e+06,3.054015e+06,2.336426e+06,2.510610e+06,0.021582,0.0
min,1.000000,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
25%,11.000000,1.335032e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
50%,14.000000,8.137560e+04,1.856900e+04,0.000000e+00,1.192711e+05,2.221441e+05,0.000000,0.0
75%,17.000000,2.226441e+05,1.702941e+05,2.101716e+05,8.962267e+05,1.201403e+06,0.000000,0.0
max,20.000000,1.000000e+07,3.893942e+07,3.894623e+07,4.148270e+07,4.148270e+07,1.000000,0.0


In [10]:
df.shape

(500000, 11)

### What is the distribution of the outcome? 

In [11]:
# Your response here - outcome = isFraud 
df['isFraud'].value_counts().plot(kind='bar')
#plt.ylim(0,110000)

In [12]:
# expected:
df['isFraud'].value_counts()

0    499767
1       233
Name: isFraud, dtype: int64

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [15]:
# Your code here
df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [16]:
df.isna().sum() # no nans 

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [20]:
# too many dimensions - lets drop:
# is flagged fraud - not disciptive not outcome
df.drop(columns=['isFlaggedFraud'], inplace=True)
# steps holds no relevant data
df.drop("step", axis=1, inplace=True)

In [21]:
# check out object columns
# convert to dummy to do regression
# nameOrig and nameDest seem to have a kind of ID, unique values, therefore we drop
df.drop(columns=['nameOrig', 'nameDest'], inplace=True)

In [23]:
dummies = pd.get_dummies(df, columns = ['type'], drop_first=True)

In [24]:
dummies.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,1,0
1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,1,0
2,181.00,181.0,0.00,0.0,0.0,1,0,0,0,1
3,181.00,181.0,0.00,21182.0,0.0,1,1,0,0,0
4,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,1,0


### Run a logisitc regression classifier and evaluate its accuracy.

In [27]:
from sklearn.model_selection import train_test_split

y = dummies[['isFraud']]
X = dummies.drop(columns=['isFraud'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12)

In [28]:
# Your code here
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 

logreg = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr')
 
logreg.fit(X_train, y_train)

C:\Users\fabia hnt\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2', random_state=0,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [29]:
y_pred = logreg.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score

# Logistic regression (same as used in other lab)
print(f"Confusion matrix = \n {confusion_matrix(y_test, y_pred)}")
print(f"precision score = {precision_score(y_test, y_pred)}")
print(f"recall = {recall_score(y_test, y_pred)}")
print(f"f1 score = {f1_score(y_test, y_pred)}")
print(f"accuracy = {accuracy_score(y_test, y_pred)}")

Confusion matrix = 
 [[164912     13]
 [    26     49]]
precision score = 0.7903225806451613
recall = 0.6533333333333333
f1 score = 0.7153284671532847
accuracy = 0.9997636363636364


### Now pick a model of your choice and evaluate its accuracy.

In [33]:
# Your code here
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion = "gini", random_state = 100, max_depth=1, min_samples_leaf=5)

# Fitting the model on train set 
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=100, splitter='best')

In [34]:
y_pred_tree = tree.predict(X_test)

In [37]:
print(f"Confusion matrix = \n {confusion_matrix(y_test, y_pred_tree)}")
print(f"precision score = {precision_score(y_test, y_pred_tree)}")
print(f"recall = {recall_score(y_test, y_pred_tree)}")
print(f"f1 score = {f1_score(y_test, y_pred_tree)}")
print(f"accuracy = {accuracy_score(y_test, y_pred_tree)}")

Confusion matrix = 
 [[164925      0]
 [    73      2]]
precision score = 1.0
recall = 0.02666666666666667
f1 score = 0.05194805194805195
accuracy = 0.9995575757575758


### Which model worked better and how do you know?

###### Your response here
accuracy of both models are very similar. 
according to confusion matrix, I have less type 1 and type 2 errors in the logistic regression. I would therefore suggest it as a better option